## MTN Nigeria Customer Churn Prediction – Data Preparation (Customer-Level)

**Business Objective**  
Predict customers at high risk of churning so MTN Nigeria can proactively design retention interventions (better bundles, network quality fixes, loyalty rewards, targeted offers) → protect & grow revenue in a very competitive telecom market.

**Notebook Goal**  
The primary objective of this notebook is to transform the raw transactional-level dataset (multiple rows per customer due to multiple devices/plans) into a clean, customer-level dataset suitable for exploratory data analysis and predictive modeling.  

In this notebook I aim to:
- Master pandas groupby + aggregation patterns for real-world transactional telecom data
- Practice thoughtful feature engineering that respects business context
- Turn messy multi-row-per-customer data into a clean, ML-ready customer-level table
- Build habits of writing self-documenting code + markdown explanations
- Create features I believe will matter for churn prediction (based on telecom domain knowledge: satisfaction, usage gaps, plan type, tenure, spend patterns)

**Dataset Source**  
Kaggle: [MTN Nigeria Customer Churn](https://www.kaggle.com/datasets/oluwademiladeadeniyi/mtn-nigeria-customer-churn)  
Original data: transaction-level (multiple purchases per customer)

**Table of Contents**
1. [Load & Inspect Raw Data](#1-load--inspect-raw-data)
2. [Feature Extraction from Subscription Plan](#2-feature-extraction-from-subscription-plan)
3. [Customer-Level Aggregation](#3-customer-level-aggregation)
4. [Derived Business Features](#4-derived-business-features)
5. [Final Checks & Save](#5-final-checks--save)

**Setup & Libraries Imports**

In [1]:
import pandas as pd
import numpy as np
import re
import os

### Load & Inspect Raw Data
<a id="1-load--inspect-raw-data"></a>

In [2]:
data = pd.read_csv(r'C:\Users\KOLADE\OneDrive\Documents\AkoladeDSJourney\MTN-Nigeria-Customer-Churn\data\raw\mtn_customer_churn.csv')

df = data.copy() # make a copy to preserve the original datasets
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Customer ID                974 non-null    object 
 1   Full Name                  974 non-null    object 
 2   Date of Purchase           974 non-null    object 
 3   Age                        974 non-null    int64  
 4   State                      974 non-null    object 
 5   MTN Device                 974 non-null    object 
 6   Gender                     974 non-null    object 
 7   Satisfaction Rate          974 non-null    int64  
 8   Customer Review            974 non-null    object 
 9   Customer Tenure in months  974 non-null    int64  
 10  Subscription Plan          974 non-null    object 
 11  Unit Price                 974 non-null    int64  
 12  Number of Times Purchased  974 non-null    int64  
 13  Total Revenue              974 non-null    int64  

**Quick Data Quality Check**

In [3]:
print(f"Dataframe shape: {df.shape}")
df.head()

Dataframe shape: (974, 17)


,Customer ID,Full Name,Date of Purchase,Age,State,MTN Device,Gender,Satisfaction Rate,Customer Review,Customer Tenure in months,Subscription Plan,Unit Price,Number of Times Purchased,Total Revenue,Data Usage,Customer Churn Status,Reasons for Churn
0,CUST0001,Ngozi Berry,Jan-25,27,Kwara,4G Router,Male,2,Fair,2,165GB Monthly Plan,35000,19,665000,44.48,Yes,Relocation
1,CUST0002,Zainab Baker,Mar-25,16,Abuja (FCT),Mobile SIM Card,Female,2,Fair,22,12.5GB Monthly Plan,5500,12,66000,19.79,Yes,Better Offers from Competitors
2,CUST0003,Saidu Evans,Mar-25,21,Sokoto,5G Broadband Router,Male,1,Poor,60,150GB FUP Monthly Unlimited,20000,8,160000,9.64,No,NaN
3,CUST0003,Saidu Evans,Mar-25,21,Sokoto,Mobile SIM Card,Male,1,Poor,60,1GB+1.5mins Daily Plan,500,8,4000,197.05,No,NaN
4,CUST0003,Saidu Evans,Mar-25,21,Sokoto,Broadband MiFi,Male,1,Poor,60,30GB Monthly Broadband Plan,9000,15,135000,76.34,No,NaN


In [4]:
print(f"Duplicates data: {df.duplicated().sum()}\n")
print(f"Missing values:\n{df.isnull().sum()}")

Duplicates data: 0

Missing values:
Customer ID                    0
Full Name                      0
Date of Purchase               0
Age                            0
State                          0
MTN Device                     0
Gender                         0
Satisfaction Rate              0
Customer Review                0
Customer Tenure in months      0
Subscription Plan              0
Unit Price                     0
Number of Times Purchased      0
Total Revenue                  0
Data Usage                     0
Customer Churn Status          0
Reasons for Churn            690
dtype: int64


In [5]:
df[df["Reasons for Churn"].isnull()]['Customer Churn Status'].value_counts()

Customer Churn Status
No    690
Name: count, dtype: int64

- There is no duplicated data
- The missing value in `Reasons for Churn` is expected - Since customer didn't churn, there should be no reason for churn

#### Key Descriptive Statistics

**Numerical Features**

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,974.0,48.043121,17.764307,16.00,32.0000,49.00,63.7500,80.0
Satisfaction Rate,974.0,2.947639,1.384219,1.00,2.0000,3.00,4.0000,5.0
Customer Tenure in months,974.0,31.422998,17.191256,1.00,17.0000,31.00,47.0000,60.0
Unit Price,974.0,19196.663244,25586.726985,350.00,5500.0000,14500.00,24000.0000,150000.0
Number of Times Purchased,974.0,10.564682,5.709427,1.00,5.0000,11.00,15.0000,20.0
Total Revenue,974.0,204669.609856,324785.499316,350.00,33000.0000,108000.00,261000.0000,3000000.0
Data Usage,974.0,99.304764,57.739511,0.82,47.6375,103.33,149.6975,200.0


**Categorical Features**

In [7]:
df.describe(include='O')

,Customer ID,Full Name,Date of Purchase,State,MTN Device,Gender,Customer Review,Subscription Plan,Customer Churn Status,Reasons for Churn
count,974,974,974,974,974,974,974,974,974,284
unique,496,484,3,35,4,2,5,21,2,7
top,CUST0003,Halima Walker,Feb-25,Osun,Mobile SIM Card,Female,Very Good,60GB Monthly Broadband Plan,No,High Call Tarriffs
freq,3,5,450,43,301,495,212,81,690,54


### Feature Extraction from Subscription Plan
<a id="2-feature-extraction-from-subscription-plan"></a>

The unique values show patterns:

- Plan_type: Categorize as 'Daily', '2-Day', 'Monthly', '3-Month', 'Yearly' based on keywords.
- GB_purchased: Parse the data amount (convert MB to GB, TB to GB; ignore extras like "+10mins" or "FUP" for now—FUP plans could have a flag if needed).
- Total_GB_purchased: GB_purchased * Number of Times Purchased (proxies total consumption intent).

In [8]:
# Function to extract GB_purchased (handles GB, MB, TB)
def extract_gb(plan):
    # Find numeric value and unit
    match = re.search(r'(\d+\.?\d*)\s?(GB|MB|TB)', plan, re.IGNORECASE)
    if match:
        value = float(match.group(1))
        unit = match.group(2).upper()
        if unit == 'MB':
            return value / 1024  # Convert to GB
        elif unit == 'TB':
            return value * 1024  # Convert to GB
        return value
    return np.nan  # If no match (rare)

In [9]:
# Function for Plan_type
def extract_plan_type(plan):
    if 'Daily' in plan:
        return 'Daily'
    elif '2-Day' in plan:
        return '2-Day'
    elif '3-Month' in plan:
        return '3-Month'
    elif 'Yearly' in plan:
        return 'Yearly'
    elif 'Monthly' in plan:
        return 'Monthly'
    return 'Other'  # Catch-all

In [10]:
# Apply extractions
df['GB_purchased'] = df['Subscription Plan'].apply(extract_gb)
df['Plan_type'] = df['Subscription Plan'].apply(extract_plan_type)
df['Total_GB_purchased'] = df['GB_purchased'] * df['Number of Times Purchased']

In [13]:
df[['Subscription Plan', 'GB_purchased', 'Plan_type', 'Total_GB_purchased']].head(10)

,Subscription Plan,GB_purchased,Plan_type,Total_GB_purchased
0,165GB Monthly Plan,165.0,Monthly,3135.0
1,12.5GB Monthly Plan,12.5,Monthly,150.0
2,150GB FUP Monthly Unlimited,150.0,Monthly,1200.0
3,1GB+1.5mins Daily Plan,1.0,Daily,8.0
4,30GB Monthly Broadband Plan,30.0,Monthly,450.0
5,10GB+10mins Monthly Plan,10.0,Monthly,90.0
6,25GB Monthly Plan,25.0,Monthly,400.0
7,7GB Monthly Plan,7.0,Monthly,35.0
8,1.5TB Yearly Broadband Plan,1536.0,Yearly,3072.0
9,65GB Monthly Plan,65.0,Monthly,390.0


In [14]:
df['Plan_type'].value_counts()

Plan_type
Monthly    777
2-Day       94
Daily       53
Yearly      25
3-Month     25
Name: count, dtype: int64

In [15]:
df['GB_purchased'].describe()

count     974.000000
mean      127.723470
std       250.973553
min         0.488281
25%        12.500000
50%        60.000000
75%       150.000000
max      1536.000000
Name: GB_purchased, dtype: float64

### Customer-Level Aggregation
<a id="3-customer-level-aggregation"></a>

**Why Aggregate?**  
The original dataset is **transaction-level** each row represents one purchase/device/subscription per customer (multiple rows per `Customer ID`).  

Churn, however, is a **customer-level outcome** (a customer either churns or doesn't).  

To build a meaningful predictive model and derive business insights:  
- We need **one row per unique customer**  
- Stable attributes (age, gender, state) → take once  
- Behavioral signals (revenue, data usage, purchases, devices) → aggregate meaningfully  
- Churn label & reasons → preserve at customer level for diagnosis  

This aggregation transforms the data from **transactional** → **customer-centric**, aligning perfectly with how MTN would actually intervene on churn (targeting individual customers, not individual purchases).

In [16]:
customer_df = (
    df.groupby('Customer ID')
      .agg(
          # === Demographics (stable across rows) ===
          Age=('Age', 'first'),
          State=('State', 'first'),
          Gender=('Gender', 'first'),
          
          # === Device & product diversity ===
          Device_Count=('MTN Device', 'nunique'),
          
          # === Purchase behaviour ===
          Active_Months=('Date of Purchase', 'nunique'),
          Total_Purchases=('Number of Times Purchased', 'sum'),
          Avg_Unit_Price=('Unit Price', 'mean'),
          Total_Revenue=('Total Revenue', 'sum'),
          
          # === Experience & satisfaction ===
          Avg_Satisfaction_Rate=('Satisfaction Rate', 'mean'),
          Primary_Review=('Customer Review', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
          
          # === Tenure & loyalty ===
          Customer_Tenure_Months=('Customer Tenure in months', 'max'),
          
          # === Data consumption behavior ===
          Avg_Data_Usage_GB=('Data Usage', 'mean'),
          Total_Data_Usage_GB=('Data Usage', 'sum'),
          Avg_GB_Purchased=('GB_purchased', 'mean'),
          Total_GB_Purchased=('Total_GB_purchased', 'sum'),
          Primary_Plan_Type=('Plan_type', lambda x: x.mode().iloc[0] if not x.empty else None),
          
          # === Target & explainability ===
          Churn_Status=('Customer Churn Status', 'max'),
          Reasons_for_Churn=(
              'Reasons for Churn',
              lambda x: (
                  ', '.join(pd.Series(x.dropna().str.strip()).unique())
                  if x.notna().any()
                  else None
              )
          )
      )
      .reset_index()
)

# Create clean binary target
customer_df['Churn'] = (customer_df['Churn_Status'] == 'Yes').astype(int)

print("Customer-level dataset created with shape:", customer_df.shape)

Customer-level dataset created with shape: (496, 20)


In [17]:
customer_df.head()

,Customer ID,Age,State,Gender,Device_Count,Active_Months,Total_Purchases,Avg_Unit_Price,Total_Revenue,Avg_Satisfaction_Rate,Primary_Review,Customer_Tenure_Months,Avg_Data_Usage_GB,Total_Data_Usage_GB,Avg_GB_Purchased,Total_GB_Purchased,Primary_Plan_Type,Churn_Status,Reasons_for_Churn,Churn
0,CUST0001,27,Kwara,Male,1,1,19,35000.000000,665000,2.0,Fair,2,44.480000,44.48,165.000000,3135.0,Monthly,Yes,Relocation,1
1,CUST0002,16,Abuja (FCT),Female,1,1,12,5500.000000,66000,2.0,Fair,22,19.790000,19.79,12.500000,150.0,Monthly,Yes,Better Offers from Competitors,1
2,CUST0003,21,Sokoto,Male,3,1,31,9833.333333,299000,1.0,Poor,60,94.343333,283.03,60.333333,1658.0,Monthly,No,None,0
3,CUST0004,36,Gombe,Female,1,1,9,4500.000000,40500,1.0,Poor,14,92.720000,92.72,10.000000,90.0,Monthly,No,None,0
4,CUST0005,57,Oyo,Male,1,1,16,9000.000000,144000,3.0,Good,53,42.920000,42.92,25.000000,400.0,Monthly,No,None,0


**Quick Data Quality Checks After Aggregation**

In [18]:
print(f"Missing values:\n{customer_df.isna().sum()}")

Missing values:
Customer ID                 0
Age                         0
State                       0
Gender                      0
Device_Count                0
Active_Months               0
Total_Purchases             0
Avg_Unit_Price              0
Total_Revenue               0
Avg_Satisfaction_Rate       0
Primary_Review              0
Customer_Tenure_Months      0
Avg_Data_Usage_GB           0
Total_Data_Usage_GB         0
Avg_GB_Purchased            0
Total_GB_Purchased          0
Primary_Plan_Type           0
Churn_Status                0
Reasons_for_Churn         350
Churn                       0
dtype: int64


### Derived Business Features
<a id="4-derived-business-features"></a>

In [19]:
customer_df['Usage_vs_Purchased_Ratio'] = (
    customer_df['Total_Data_Usage_GB'] / customer_df['Total_GB_Purchased'].replace(0, np.nan)
)

customer_df['Revenue_per_Purchase'] = (
    customer_df['Total_Revenue'] / customer_df['Total_Purchases'].replace(0, np.nan)
)

customer_df['Tenure_Bucket'] = pd.cut(
    customer_df['Customer_Tenure_Months'],
    bins=[0, 6, 12, 24, 36, np.inf],
    labels=['0-6', '7-12', '13-24', '25-36', '37+'],
    include_lowest=True
)

In [21]:
customer_df.head()

,Customer ID,Age,State,Gender,Device_Count,Active_Months,Total_Purchases,Avg_Unit_Price,Total_Revenue,Avg_Satisfaction_Rate,...,Total_Data_Usage_GB,Avg_GB_Purchased,Total_GB_Purchased,Primary_Plan_Type,Churn_Status,Reasons_for_Churn,Churn,Usage_vs_Purchased_Ratio,Revenue_per_Purchase,Tenure_Bucket
0,CUST0001,27,Kwara,Male,1,1,19,35000.000000,665000,2.0,...,44.48,165.000000,3135.0,Monthly,Yes,Relocation,1,0.014188,35000.00000,0-6
1,CUST0002,16,Abuja (FCT),Female,1,1,12,5500.000000,66000,2.0,...,19.79,12.500000,150.0,Monthly,Yes,Better Offers from Competitors,1,0.131933,5500.00000,13-24
2,CUST0003,21,Sokoto,Male,3,1,31,9833.333333,299000,1.0,...,283.03,60.333333,1658.0,Monthly,No,None,0,0.170706,9645.16129,37+
3,CUST0004,36,Gombe,Female,1,1,9,4500.000000,40500,1.0,...,92.72,10.000000,90.0,Monthly,No,None,0,1.030222,4500.00000,13-24
4,CUST0005,57,Oyo,Male,1,1,16,9000.000000,144000,3.0,...,42.92,25.000000,400.0,Monthly,No,None,0,0.107300,9000.00000,37+


In [24]:
# Quick sanity check
print("Customer-level shape:", customer_df.shape)
print(f"\nChurn rate: {customer_df['Churn'].mean() * 100:.2f}%")
print(f"\nMissing values:\n{customer_df.isnull().sum()}")

Customer-level shape: (496, 23)

Churn rate: 29.44%

Missing values:
Customer ID                   0
Age                           0
State                         0
Gender                        0
Device_Count                  0
Active_Months                 0
Total_Purchases               0
Avg_Unit_Price                0
Total_Revenue                 0
Avg_Satisfaction_Rate         0
Primary_Review                0
Customer_Tenure_Months        0
Avg_Data_Usage_GB             0
Total_Data_Usage_GB           0
Avg_GB_Purchased              0
Total_GB_Purchased            0
Primary_Plan_Type             0
Churn_Status                  0
Reasons_for_Churn           350
Churn                         0
Usage_vs_Purchased_Ratio      0
Revenue_per_Purchase          0
Tenure_Bucket                 0
dtype: int64


### Final Checks & Save
<a id="5-final-checks--save"></a>

In [25]:
processed_folder = r"C:\Users\KOLADE\OneDrive\Documents\AkoladeDSJourney\MTN-Nigeria-Customer-Churn\data\processed"

os.makedirs(processed_folder, exist_ok=True)

save_path = os.path.join(processed_folder, "mtn_customer_level_churn.csv")

customer_df.to_csv(save_path, index=False)
print(f"Saved to: {save_path}")

Saved to: C:\Users\KOLADE\OneDrive\Documents\AkoladeDSJourney\MTN-Nigeria-Customer-Churn\data\processed\mtn_customer_level_churn.csv
